In [ ]:
from hsflfm.processing import Aligner, world_frame_to_pixel
from hsflfm.analysis import ResultPlotter
from hsflfm.util import (
    MetadataManager,
    matmul,
)
from hsflfm.calibration import FLF_System
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
specimen_numbers = MetadataManager.all_specimen_numbers()

In [ ]:
num = specimen_numbers[10]
data_manager = MetadataManager(num)
aligner = Aligner(num)

A_base, s_base = aligner.run_base_alignment()
A, s = aligner.run_strike1_alignment()

mp_base = aligner.move_points_to_mesh(
    A_base, s_base, aligner.point_camera_locations
)
mp = aligner.move_points_to_mesh(A, s, aligner.point_camera_locations)

In [ ]:
# Display mesh with points before and after fine alignment
p = np.concatenate((mp_base, mp))
values = ["blue"] * len(mp_base) + ["red"] * len(mp)

ResultPlotter.plot_mesh_with_points(
    points=p, point_values=values, use_display_mesh=True, opacity=0.5
)

In [ ]:
# display base and refined coordinate system on ant images
def show_coord_system(cam_number=0):
    system = FLF_System(data_manager.calibration_filename)
    dark_image = data_manager.get_start_images(strike_number=1)[cam_number]
    light_image = data_manager.light_calibration_images[cam_number]

    scale = 0.5
    points = [[0, 0, 0], [scale, 0, 0], [0, scale, 0]]
    cam_points = matmul(np.linalg.inv(A), points)
    pixels = (
        np.asarray([world_frame_to_pixel(system, i, camera=cam_number) for i in cam_points])
        .squeeze()
        .T
    )
    cam_points_base = matmul(np.linalg.inv(A_base), points)
    pixels_base = (
        np.asarray(
            [world_frame_to_pixel(system, i, camera=cam_number) for i in cam_points_base]
        )
        .squeeze()
        .T
    )

    fig, (ax0, ax1) = plt.subplots(1, 2)
    ax0.imshow(dark_image, cmap="gray")
    ax1.imshow(light_image, cmap="gray")

    for ax in (ax0, ax1):
        ax.plot(
            pixels_base[1, :2],
            pixels_base[0, :2],
            "-",
            color="blue",
            label="base alignment",
        )
        ax.plot(
            [pixels_base[1, 0], pixels_base[1, 2]],
            [pixels_base[0, 0], pixels_base[0, 2]],
            color="blue",
        )

        ax.plot(pixels[1, :2], pixels[0, :2], "-", color="red")
        ax.plot(
            [pixels[1, 0], pixels[1, 2]],
            [pixels[0, 0], pixels[0, 2]],
            color="red",
            label="fine alignment",
        )

        ax.set_xticks([])
        ax.set_yticks([])
    ax1.legend()
    fig.suptitle(num + ', ' + str(i))
    plt.tight_layout()


show_coord_system()

In [ ]:
# then color separately to check for obvious shifts between the images
cam_number = 2
dark_image = data_manager.get_start_images(strike_number=1)[cam_number]
light_image = data_manager.light_calibration_images[cam_number]

c_img = np.zeros((dark_image.shape[0], dark_image.shape[1], 3))
c_img[:, :, 0] = light_image
c_img[:, :, 2] = dark_image * 5

c_img = (c_img - np.min(c_img)) / (np.max(c_img) - np.min(c_img))

plt.imshow(c_img)
plt.gca().set_xticks([])
plt.gca().set_yticks([])

In [ ]:
# Possible samples of concern
# 20220427_OB_4
# 20240427_OB_5
# 20240417_OB_2
# 20240417_OB_1
# 20240502_OB_2
# 20240503_OB_4
# 20240503_OB_5

# Good examples:
# 20240430_OB_2

In [ ]:
for i, num in enumerate(specimen_numbers):
    data_manager = MetadataManager(num)
    aligner = Aligner(num)

    A_base, s_base = aligner.run_base_alignment()
    A, s = aligner.run_strike1_alignment()

    show_coord_system()

    plt.show()